In [1]:
from datasets import load_from_disk
 
# Load dataset from the Disk
dataset = load_from_disk("APN_dataset")
 

# Add an id column to the dataset
dataset = dataset.add_column("id", range(len(dataset)))
print (f"Dataset: {dataset[1]}") 
# split dataset into a 10% test set
dataset = dataset.train_test_split(test_size=0.1)
 
# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")


/home/mec-srv/miniconda3/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset: {'anchor': 'بله، به نقطه تاس۱۰ توجه کنید: «امکان ضمانت متقابل انواع وام\u200cها در بانک ملت وجود ندارد».', 'positive': 'آیا امکان ضمانت متقابل در وام\u200cهای مختلف در بانک ملت وجود دارد؟', 'negetive': 'نحوه محاسبه توان بازپرداخت چگونه است؟', 'id': 1}


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1024.00ba/s]


123317

In [2]:
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets
 
model_id = "halong_finetuned_frezzed"  

# model_id = "hiieu/halong_embedding" 

matryoshka_dimensions = [768, 512, 256, 128, 64] 
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)
 
# load test dataset
test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])
 
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
)  # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
)  # Our queries (qid => question)
 
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]
 
 
matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)
 
# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

No sentence-transformers model found with name halong_finetuned_frezzed. Creating a new one with mean pooling.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at halong_finetuned_frezzed and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Generating train split: 69 examples [00:00, 45184.54 examples/s]
Generating train split: 617 examples [00:00, 58422.56 examples/s]


In [3]:
# Evaluate the model
results = evaluator(model)
 
# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.39444353944796706
dim_512_cosine_ndcg@10: 0.38574244041851047
dim_256_cosine_ndcg@10: 0.35502801651499005
dim_128_cosine_ndcg@10: 0.3725521848226817
dim_64_cosine_ndcg@10: 0.35926891916256354


In [4]:
# Evaluate the model
results = evaluator(model)
 
# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.39444353944796706
dim_512_cosine_ndcg@10: 0.38574244041851047
dim_256_cosine_ndcg@10: 0.35502801651499005
dim_128_cosine_ndcg@10: 0.3725521848226817
dim_64_cosine_ndcg@10: 0.35926891916256354


In [5]:
from sentence_transformers import SentenceTransformerModelCardData, SentenceTransformer
 

 
# load model with SDPA for using Flash Attention 2
model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="multilingual",
        license="apache-2.0",
        model_name="Halang",
    ),
)

No sentence-transformers model found with name halong_finetuned_frezzed. Creating a new one with mean pooling.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at halong_finetuned_frezzed and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
 
matryoshka_dimensions = [768, 512, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [7]:
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=20,  
    early_stopping_threshold=0.001  
)


In [8]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
 
# load train dataset again
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
 
# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="halong_finetuned_APN", # output directory and hugging face model ID
    num_train_epochs=300,                         # number of epochs
    per_device_train_batch_size=32,             # train batch size
    gradient_accumulation_steps=16,             # for a global batch size of 512
    per_device_eval_batch_size=16,              # evaluation batch size
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    tf32=True,                                  # use tf32 precision
    bf16=True,                                  # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                      # evaluate after each epoch
    save_strategy="epoch",                      # save after each epoch
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
    greater_is_better=True
    
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
from sentence_transformers import SentenceTransformerTrainer
 
trainer = SentenceTransformerTrainer(
    model=model,
    args=args, 
    train_dataset=train_dataset.select_columns(
        ["anchor" , "positive" , "negetive"]
    ), 
    loss=train_loss,
    evaluator=evaluator,callbacks=[early_stopping]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
# start training, 
trainer.train()
 


Epoch,Training Loss,Validation Loss,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,No log,No log,0.217391,0.347826,0.434783,0.637681,0.217391,0.115942,0.086957,0.063768,0.217391,0.347826,0.434783,0.637681,0.394703,0.321871,0.337919,0.202899,0.362319,0.420290,0.637681,0.202899,0.120773,0.084058,0.063768,0.202899,0.362319,0.420290,0.637681,0.392909,0.319053,0.335141,0.188406,0.304348,0.420290,0.579710,0.188406,0.101449,0.084058,0.057971,0.188406,0.304348,0.420290,0.579710,0.353764,0.285507,0.304095,0.217391,0.304348,0.391304,0.594203,0.217391,0.101449,0.078261,0.059420,0.217391,0.304348,0.391304,0.594203,0.367462,0.300075,0.316170,0.202899,0.318841,0.376812,0.579710,0.202899,0.106280,0.075362,0.057971,0.202899,0.318841,0.376812,0.579710,0.360500,0.294531,0.310899,0.360500
2,No log,No log,0.217391,0.347826,0.434783,0.652174,0.217391,0.115942,0.086957,0.065217,0.217391,0.347826,0.434783,0.652174,0.398560,0.322976,0.338323,0.202899,0.362319,0.420290,0.623188,0.202899,0.120773,0.084058,0.062319,0.202899,0.362319,0.420290,0.623188,0.388084,0.316879,0.334309,0.202899,0.318841,0.420290,0.579710,0.202899,0.106280,0.084058,0.057971,0.202899,0.318841,0.420290,0.579710,0.360586,0.294421,0.313626,0.231884,0.304348,0.391304,0.608696,0.231884,0.101449,0.078261,0.060870,0.231884,0.304348,0.391304,0.608696,0.377636,0.309495,0.324688,0.202899,0.318841,0.362319,0.594203,0.202899,0.106280,0.072464,0.059420,0.202899,0.318841,0.362319,0.594203,0.366192,0.297970,0.313531,0.366192
3,No log,No log,0.217391,0.347826,0.449275,0.666667,0.217391,0.115942,0.089855,0.066667,0.217391,0.347826,0.449275,0.666667,0.405885,0.328232,0.342802,0.231884,0.347826,0.434783,0.666667,0.231884,0.115942,0.086957,0.066667,0.231884,0.347826,0.434783,0.666667,0.410205,0.333972,0.348385,0.202899,0.318841,0.391304,0.608696,0.202899,0.106280,0.078261,0.060870,0.202899,0.318841,0.391304,0.608696,0.370406,0.299189,0.316530,0.217391,0.333333,0.405797,0.608696,0.217391,0.111111,0.081159,0.060870,0.217391,0.333333,0.405797,0.608696,0.376396,0.307241,0.323380,0.188406,0.318841,0.420290,0.579710,0.188406,0.106280,0.084058,0.057971,0.188406,0.318841,0.420290,0.579710,0.3

TrainOutput(global_step=76, training_loss=30.202686811748304, metrics={'train_runtime': 222.146, 'train_samples_per_second': 833.236, 'train_steps_per_second': 1.35, 'total_flos': 0.0, 'train_loss': 30.202686811748304, 'epoch': 38.0})

In [12]:
# save the best model
trainer.save_model()
 


In [11]:
from sentence_transformers import SentenceTransformer
 
fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluate the model
results = evaluator(fine_tuned_model)
 

# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.7340257807397909
dim_512_cosine_ndcg@10: 0.7632895558112947
dim_256_cosine_ndcg@10: 0.7748767678236386
dim_128_cosine_ndcg@10: 0.8145157578527988
dim_64_cosine_ndcg@10: 0.7742869662818314
